<a href="https://colab.research.google.com/github/Jocelyn-Griser/ShakespeareBot/blob/main/Obscured_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title Setup

import cv2
from matplotlib import pyplot as plt
import numpy as np
import time
import random
from PIL import Image

!pip install -q gradio
import gradio as gr

def randomOverlay(imagename):
  notempty = cv2.subtract(imagename,imagename)


  biggest_x = int(notempty.shape[0]/20)
  biggest_y = int(notempty.shape[1]/20)

  for i in range(2000):

    boxi_x=random.randrange(0,notempty.shape[0])
    boxi_dx=random.randrange(0,int(biggest_x))
    if boxi_x+boxi_dx>notempty.shape[0]:
      boxi_dx=notempty.shape[0]-boxi_x

    boxi_y=random.randrange(0,notempty.shape[1])
    boxi_dy=random.randrange(0,int(biggest_y))
    if boxi_y+boxi_dy>notempty.shape[1]:
      boxi_dy=notempty.shape[1]-boxi_y

    boxi_r=random.randrange(0,180)
    boxi_g=random.randrange(0,180)
    boxi_b=random.randrange(0,180)

    # print(boxi_x,boxi_dx,boxi_y,boxi_dy,boxi_rgb,boxi_value)

    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,0] += boxi_r
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,1] += boxi_g
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,2] += boxi_b

    notempty[notempty>255] = 255

  # plt.figure(figsize=(8,8))
  # plt.imshow(cv2.cvtColor(notempty, cv2.COLOR_BGR2RGB))
  # plt.axis('off')
  # plt.show()

  return notempty

def semiRandomOverlay(imagename):
  notempty = cv2.subtract(imagename,imagename)


  x_or_y = int(notempty.shape[0]/35)
  y_or_x = int(notempty.shape[1]/35)

  for i in range(2000):

    xy = random.randint(0,1)
    if (xy==0):
      x = x_or_y
      y = y_or_x
    else:
      x = y_or_x
      y = x_or_y

    boxi_x=random.randrange(0,notempty.shape[0])
    boxi_dx=x
    if boxi_x+boxi_dx>notempty.shape[0]:
      boxi_dx=notempty.shape[0]-boxi_x

    boxi_y=random.randrange(0,notempty.shape[1])
    boxi_dy=y
    if boxi_y+boxi_dy>notempty.shape[1]:
      boxi_dy=notempty.shape[1]-boxi_y

    boxi_r=random.randrange(0,180)
    boxi_g=random.randrange(0,180)
    boxi_b=random.randrange(0,180)

    # print(boxi_x,boxi_dx,boxi_y,boxi_dy,boxi_rgb,boxi_value)

    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,0] += boxi_r
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,1] += boxi_g
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,2] += boxi_b

    notempty[notempty>255] = 255

  # plt.figure(figsize=(8,8))
  # plt.imshow(cv2.cvtColor(notempty, cv2.COLOR_BGR2RGB))
  # plt.axis('off')
  # plt.show()

  return notempty, x, y


def ObscureImage(imagepath,
                rotation1, flip1, overlay1divisor, addWeight1,
                addRandom, randomDivisor, randomWeight,
                rotation2, flip2, overlay2divisor, addWeight2,
                cannythresh1,cannythresh2
                ):


  startTime = time.perf_counter()

  orig = cv2.imread(imagepath)

  cv2.imwrite("Original.png",orig)

  # Step 1 - Sobel XY of greyscale
  step1_0 = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
  step1 = cv2.Sobel(src=step1_0, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5)

  cv2.imwrite("step1.png",step1)
  step1 = cv2.imread("step1.png")

  # Step 2 - Find Sift Keypoints
  sift = cv2.xfeatures2d.SIFT_create()
  keypoints_sift, descriptors_sift = sift.detectAndCompute(orig, None)

  step2 = cv2.drawKeypoints(orig, keypoints_sift, 0,  flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT) #this is for illustrative purposes, not for processing

  cv2.imwrite("step2.png",step2)
  step2 = cv2.imread("step2.png")

  # Step 3 - Create overlay: draw keypoints over sobel, rotate, decrease intensity
  step3_1 = cv2.drawKeypoints(step1, keypoints_sift, 0, flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT)

  if rotation1==90 or rotation1==270:
    if rotation1==90:
      step3_2 = cv2.rotate(step3_1,cv2.ROTATE_90_CLOCKWISE)
    else:
      step3_2 = cv2.rotate(step3_1,cv2.ROTATE_90_COUNTERCLOCKWISE)
    step3_2 = cv2.resize(step3_2, (orig.shape[1],orig.shape[0]))
  elif rotation1==180:
    step3_2 = cv2.rotate(step3_1,cv2.ROTATE_180)
  elif rotation1==0:
    step3_2=step3_1
  else:
    print("Error: Cannot rotate by ",rotation1,"degrees; defaulting to 180")
    step3_2 = cv2.rotate(step3_1,cv2.ROTATE_180)

  if flip1=="X":
    step3_3=cv2.flip(step3_2,0)
  elif flip1=="Y":
    step3_3=cv2.flip(step3_2,1)
  elif flip1=="None":
    step3_3=step3_2
  else:
    print("Error:\"",flip1,"\" is not a valid flip parameter; defaulting to no flip")
    step3_3=step3_2

  cv2.imwrite("step3_3.png",step3_3)
  step3_3 = cv2.imread("step3_3.png")

  step3 = step3_3/overlay1divisor

  cv2.imwrite("step3.png",step3)
  step3 = cv2.imread("step3.png")

  # Step 4 - Add Step3 to orig
  step4 = cv2.addWeighted(step3, addWeight1, orig, 1-addWeight1, 0.0, 0)

  cv2.imwrite("step4.png",step4)
  step4 = cv2.imread("step4.png")

  # Step 5 - Add random overlay if so desired
  if (addRandom=='Random' or addRandom=='Semi-Random'):
    if (addRandom=='Semi-Random'):
      step5_0, semix, semiy = semiRandomOverlay(step4) #it doesn't actually matter what we plug-in here, as long as it's the same shape as the resulting image
    else:
      step5_0 = randomOverlay(step4) #it doesn't actually matter what we plug-in here, as long as it's the same shape as the resulting image

    cv2.imwrite("step5_0.png",step5_0)
    step5_0 = cv2.imread("step5_0.png")

    step5_1 = step5_0/randomDivisor
    cv2.imwrite("step5_1.png",step5_1)
    step5_1 = cv2.imread("step5_1.png")

    step5 = cv2.addWeighted(step5_1, randomWeight, step4, 1-randomWeight, 0.0, 0)

    cv2.imwrite("step5.png",step5)
    step5 = cv2.imread("step5.png")

  else: # no random overlay
    step5_0 = cv2.subtract(step4,step4)
    step5_0 = cv2.putText(step5_0, 'No Random Overlay', (50,50), cv2.FONT_HERSHEY_SIMPLEX,
                          1, (255,255,255),2,cv2.LINE_AA)
    randomWeight=0
    step5= step4

 # step 6 - Find canny of Step4 & rotate
  step6 = cv2.Canny(image=step5, threshold1=cannythresh1, threshold2= cannythresh2)

  if rotation2==90 or rotation2==270:
    if rotation2==90:
      step6 = cv2.rotate(step6,cv2.ROTATE_90_CLOCKWISE)
    else:
      step6 = cv2.rotate(step6,cv2.ROTATE_90_COUNTERCLOCKWISE)
    step6 = cv2.resize(step6, (orig.shape[1],orig.shape[0]))
  elif rotation2==180:
    step6 = cv2.rotate(step6,cv2.ROTATE_180)
  elif rotation2==0:
    step6=step6
  else:
    print("Cannot rotate by ",rotation2,"degrees; defaulting to 180")
    step6 = cv2.rotate(step6,cv2.ROTATE_180)

  if flip2=="X":
    step6=cv2.flip(step6,0)
  elif flip2=="Y":
    step6=cv2.flip(step6,1)
  elif flip2=="None":
    step6=step6
  else:
    print("Error:\"",flip2,"\" is not a valid flip parameter; defaulting to no flip")

  cv2.imwrite("step6_0.png",step6)
  step6_0 = cv2.imread("step6_0.png")

  step6 = step6/ overlay2divisor

  cv2.imwrite("step6.png",step6)
  step6 = cv2.imread("step6.png")

  # step 7 - Add step6 to Step4
  step7 = cv2.addWeighted(step6, addWeight2, step5, 1-addWeight2, 0.0, 0)

  cv2.imwrite("step7.png",step7)
  step7 = cv2.imread("step7.png")

  # Step 8 - Adjust brightness of step7

  step8 = step7 * (1/(1-(addWeight1+addWeight2)))
  step8[step8>255] = 255

  cv2.imwrite("step8.png", step8)
  step8=Image.open("step8.png")


  endTime = time.perf_counter()
  totalTime = endTime - startTime

  readMeFile="process.readme"
  with open(readMeFile,'w') as f:
    f.write("\nOverlay 1:")
    f.write("\n   Sobel Edge Detection")
    f.write("\n   SIFT Keypoints")
    f.write("\n   Rotation: "+str(rotation1))
    f.write("\n   Flip: "+str(flip1))
    f.write("\n   Overlay intensity: 1/"+str(overlay1divisor))
    f.write("\n   Overlay weight: "+str(addWeight1))
    f.write("\n")
    f.write("\nOverlay 2:")
    if (addRandom == 'Random' or 'Semi-Random'):
      f.write("\n   Randomly Colored Boxes")
      if (addRandom == 'Semi-Random'):
        f.write("\n   Boxes of same shape: "+str(semix)+"x"+str(semiy)+" pixels")
      f.write("\n   Overlay intensity: 1/"+str(randomDivisor))
      f.write("\n   Overlay weight: "+str(randomWeight))
      f.write("\n")
      f.write("\nOverlay 3:")
    f.write("\n   Canny Edge Detection")
    f.write("\n     Thresholds: "+str(cannythresh1)+", "+str(cannythresh2))
    f.write("\n   Rotation: "+str(rotation2))
    f.write("\n   Flip: "+str(flip2))
    f.write("\n   Overlay intensity: 1/"+str(overlay2divisor))
    f.write("\n   Overlay weight: "+str(addWeight2))


  plt.figure(figsize=(20,30))

  plt.subplot(4,3,1)
  plt.imshow(cv2.cvtColor(step3_3, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('First Overlay')

  plt.subplot(4,3,2)
  plt.imshow(cv2.cvtColor(step3, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('First Overlay Reduced')

  plt.subplot(4,3,3)
  plt.imshow(cv2.cvtColor(step4, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('First Overlay Added')

  plt.subplot(4,3,4)
  plt.imshow(cv2.cvtColor(step5_0, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Second Overlay')

  plt.subplot(4,3,5)
  plt.imshow(cv2.cvtColor(step5_1, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Second Overlay Reduced')

  plt.subplot(4,3,6)
  plt.imshow(cv2.cvtColor(step5, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Second Overlay Added')

  plt.subplot(4,3,7)
  plt.imshow(cv2.cvtColor(step6_0, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Third Overlay')

  plt.subplot(4,3,8)
  plt.imshow(cv2.cvtColor(step6, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Third Overlay Reduced')

  plt.subplot(4,3,9)
  plt.imshow(cv2.cvtColor(step7, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Third Overlay Added')

  plt.subplot(4,3,11)
  plt.imshow(np.asarray(step8))
  plt.axis('off')
  plt.title('Final Image')

  plt.savefig("process.png")
  process =Image.open("process.png")

  !rm Original.png
  !rm step1.png
  !rm step2.png
  !rm step3.png
  !rm step3_3.png
  !rm step4.png
  !rm step5.png
  !rm step5_0.png
  !rm step5_1.png
  !rm step6.png
  !rm step6_0.png
  !rm step7.png

  return step8, totalTime, readMeFile, process

In [5]:
#@title Obscured


demo = gr.Interface(
    fn=ObscureImage,
    inputs=[
        gr.Image(type='filepath',label='Input Image'),
        gr.Radio([0, 90, 180, 270], label='Overlay 1 Rotation', value=180),
        gr.Radio(['None','X','Y'], label='Overlay 1 Flip', value='None'),
        gr.Slider(1,20,step=1, label='Overlay 1 Divisor', info='The RGB values will divided by this number',value=3),
        gr.Slider(0,0.5,step=0.01, label='Overlay 1 Weight',value=0.25),
        gr.Radio(['None', 'Random', 'Semi-Random'], label='Add Random Overlay',value='Random'),
        gr.Slider(1,20,step=1, label='Random Overlay Divisor', info='The RGB values will divided by this number',value=1),
        gr.Slider(0,0.5,step=0.01, label='Random Overlay Weight',value=0.05),
        gr.Radio([0, 90, 180, 270], label='Overlay 2 Rotation', value=180),
        gr.Radio(['None','X','Y'], label='Overlay 2 Flip', value='None'),
        gr.Slider(1,20,step=1, label='Overlay 2 Divisor', info='The RGB values will divided by this number',value=3),
        gr.Slider(0,0.5,step=0.01, label='Overlay 2 Weight', value=0.05),
        gr.Slider(50,300,step=50, label='Canny: Lower Threshold', value=50),
        gr.Slider(100,350,step=50, label='Canny: Lower Threshold', value=100),

    ],
    outputs=[gr.Image(type='filepath', label="Obscured"),
             gr.Number(label='Processing Time (seconds)'),
             gr.File(label="ReadMe file"),
             gr.Image(type='filepath', label="Process")]
)
#demo.launch(debug=True)
demo.launch(share=True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3315cb029f4429f37e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
